In [1]:
import pandas as pd
import geopandas as gpd
import plotly.express as px
import os

#show me all the columns
pd.set_option('display.max_columns', None)

## Behind the scenes with geo files

Helpful Minneapolis codes:
- Minnesota is state fips 27
- Minneapolis is in Hennepin County, which is county fips 27053.

Where to download/find these GIS files for your state!
- **Schools:** [The NCES EDGE open data portal](https://data-nces.opendata.arcgis.com/) contains all sorts of helpful data, including school locations.
- **Neighborhoods:** check your city GIS site
- **Toxic Release Inventory:** Multiple years and states [available here](https://www.epa.gov/toxics-release-inventory-tri-program/tri-basic-data-files-calendar-years-1987-present)
- **Census Tigerline URLs:** [pick your vintage here](https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.html), then click FTP Archive and navigate to the shapefile you need.

In [2]:
gis_base = '../GIS'

#importing shapefiles and geojson
schools = gpd.read_file(f'{gis_base}/nces-mn-schools.geojson')

#importing from zip
hoods = gpd.read_file(f'{gis_base}/Minneapolis_Neighborhoods_4501150611105070206.zip')

#importing from url
tracts = gpd.read_file('https://www2.census.gov/geo/tiger/TIGER2024/TRACT/tl_2024_27_tract.zip')

#importing points from csv
tri = pd.read_csv(f'{gis_base}/epa-tri-2023-mn.csv')
tri = gpd.GeoDataFrame(tri, geometry=gpd.points_from_xy(tri['13. LONGITUDE'], tri['12. LATITUDE']))

In [ ]:
#exploring geodata is just like exploring any other dataframe
print('School count:',len(schools))
print(schools.dtypes)
display(schools.head())


#look the geometry!
print(schools.geometry)

#we can get the projection real quick too
print('Coordinate reference system:',schools.crs)

In [ ]:
#now you try looking at the other geodataframes crs

Cam is gonna talk about projects here for a minute. They're important friends.

In [ ]:
#quick viz
schools.plot()

## Slaying with APIs
A lot of ArcGIS maps have helpful API portals that will walk you thru querying and downloading the geojson.

Here's a video that shows you how to find that query page when you're looking at an ArcGIS map online:

Some of the data you try and pull down will clearly be accessible via API. Here's a video that shows you how to find the API page: 

In [ ]:
#let's grab some data from the Minneapolis Open Data portal
potholes = gpd.read_file("https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Public_311_2024/FeatureServer/0/query?where=TYPENAME%20%3D%20'Pothole'&outFields=*&outSR=4326&f=json")

In [ ]:
print(len(potholes))
display(potholes.head())
potholes.plot()

In [17]:
#sometimes an API requires an access key
#here's an example oh how to get Census ACS 5-yr data via the API
#and more info on the various APIs https://www.census.gov/data/developers/data-sets/acs-5year.html
#NOTE: different tables (detailed, subject, etc) have different geography levels available
def get_census_data(year,var_str,rename_cols):
    data_url = f'https://api.census.gov/data/{str(year)}/acs/acs5/subject?get={var_str}&for=tract:*&in=state:27%20county:053&key={MYKEY}'
    census_df = pd.read_json(data_url)
    new_header = census_df.iloc[0] #grab the first row for the header
    census_df = census_df[1:] #take the data less the header row
    census_df.columns = new_header #set the header row as the df header
    census_df.rename(columns=rename_cols, inplace=True) #heres where we actually rename

    #I recommend saving your API data to a file so you're not constantly hitting the API
    #census_df.to_csv(f'../data/analyzed/{str(year)}ACS5yr-hennipen-mn-tract-data.csv',index=False)
    
    return census_df

MYKEY = os.environ['census_api_key']

rename_cols = {
    'S0101_C01_001E':'pop',
    'S1901_C01_001E':'households',
    'S1901_C01_012E':'median_income',
    }

var_list = ['GEO_ID','NAME']+list(rename_cols.keys())
var_str = ','.join(var_list)
   
data_2023 = get_census_data(2023,var_str,rename_cols)

#convert our data columns to numeric
data_2023['pop'] = pd.to_numeric(data_2023['pop'],errors='coerce')
data_2023['households'] = pd.to_numeric(data_2023['households'],errors='coerce')
data_2023['median_income'] = pd.to_numeric(data_2023['median_income'],errors='coerce')

In [18]:
print(data_2023.dtypes)

0
GEO_ID           object
NAME             object
pop               int64
households        int64
median_income     int64
state            object
county           object
tract            object
dtype: object


## Spatial joins and filtering

In [ ]:
#filter one dataset by another
potholes_in_hoods = gpd.sjoin(potholes, hoods, predicate='within')

#remember, gdfs need to be in the same projection for spatial joins
potholes_4326 = potholes.to_crs(epsg=4326)
hoods_4326 = hoods.to_crs(epsg=4326)

potholes_in_hoods = gpd.sjoin(potholes_4326, hoods_4326, predicate='within')

In [ ]:
print(len(potholes))
print(len(potholes_in_hoods))
potholes_in_hoods.plot()

## Count points in polygon

## Crosswalks
Generally, demographic data like population and median income isn't calculated for custom geographies like neighborhoods. But no matter! We can calculate that sort of information for ourselves using crosswalks.

In this example, we are going to cross Census tracts with Minneapolis neighborhoods.

In [19]:
#####################
# Add census data to the tracts
#####################

#make sure we can join on the GEOID cols
data_2023['GEOID'] = data_2023['GEO_ID'].str.replace('1400000US','')

#join
tracts_data = tracts.merge(data_2023, on='GEOID', how='left')
print(len(tracts_data.loc[~tracts_data['pop'].isnull()]))

#make sure both gdfs are in the same projection and that projection uses meters or feet
tracts_data = tracts_data.to_crs(epsg=6505)
hoods = hoods.to_crs(epsg=6505)

#and we'll need to calculate the area of the tracts before and after we do the intersection so we can
#calculate the actual estimated count of people of different races in each tract segmemt.
tracts_data['pre_area'] = tracts_data['geometry'].area

#now let's do that intersection!
tracts_hoods = gpd.overlay(tracts_data, hoods, how='intersection')

#and calculate the area post intersection.  
tracts_hoods['post_area'] = tracts_hoods['geometry'].area

#from the pre and post area calculations we create a percentage
tracts_hoods['pct_area'] = tracts_hoods['post_area']/tracts_hoods['pre_area']

#and then we multiply our tract data counts by that percentage
#to get estimates per segments of the tract that falls within each hood
tracts_hoods['post_pop'] = tracts_hoods['pop'] * tracts_hoods['pct_area']
tracts_hoods['post_households'] = tracts_hoods['households'] * tracts_hoods['pct_area']
tracts_hoods['post_income'] = tracts_hoods['median_income'] * tracts_hoods['pct_area']

#groupby the neighborhood name so we can join back to our neighborhood shapes
hoods_data = tracts_hoods.groupby('BDNAME').agg({'post_pop':'sum',
                                                 'post_households':'sum',
                                                 'post_income':'mean'
                                                 }).reset_index()

329


In [20]:
hoods_data.head()

,BDNAME,post_pop,post_households,post_income
0,Armatage,4927.272657,1930.251413,32059.390183
1,Audubon Park,4795.355401,2100.987299,24809.378513
2,Bancroft,3628.632091,1503.555573,20924.895937
3,Beltrami,495.480057,198.526625,3630.264787
4,Bottineau,1697.177095,768.063281,16686.740509


## Buffers

## Calculating distance

## Exporting for viz